In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
test_transaction = pd.read_csv('test_transaction.csv')
train_identity = pd.read_csv('train_identity.csv')
test_identity = pd.read_csv('test_identity.csv')
train_transaction = pd.read_csv('train_transaction.csv')

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

calculating on which transaction happened

In [ ]:
test_transaction['day'] = np.ceil(test_transaction.TransactionDT / 86400)
train_transaction['day'] = np.ceil(train_transaction.TransactionDT / 86400)

average amount spent per day by card1

In [ ]:
avg_amt_day = test_transaction[['day', 'card1', 'TransactionAmt']].groupby(by=['day', 'card1'])['TransactionAmt'].mean().reset_index()
test_transaction['avg_amt_day'] = avg_amt_day['TransactionAmt']

avg_amt_day = train_transaction[['day', 'card1', 'TransactionAmt']].groupby(by=['day', 'card1'])['TransactionAmt'].mean().reset_index()
train_transaction['avg_amt_day'] = avg_amt_day['TransactionAmt']
del avg_amt_day

average amount spent totally by **card1**

In [ ]:
avg_amt_spent = train_transaction[['card1', 'TransactionAmt']].groupby(['card1']).sum()
avg_amt_spent = avg_amt_spent.rename(columns={'TransactionAmt':'avg_amt_spent'})
train_transaction = train_transaction.merge(avg_amt_spent, on='card1', how='left').reset_index()

avg_amt_spent = test_transaction[['card1', 'TransactionAmt']].groupby(['card1']).sum()
avg_amt_spent = avg_amt_spent.rename(columns={'TransactionAmt':'avg_amt_spent'})
test_transaction = test_transaction.merge(avg_amt_spent, on='card1', how='left').reset_index()
del avg_amt_spent

std amount spent totally by card1

In [ ]:
std_amt_spent = train_transaction[['card1', 'TransactionAmt']].groupby(['card1']).std()
std_amt_spent = std_amt_spent.rename(columns={'TransactionAmt':'std_amt_spent'})
train_transaction = train_transaction.merge(std_amt_spent, on='card1', how='left').reset_index()

In [ ]:
std_amt_spent = test_transaction[['card1', 'TransactionAmt']].groupby(['card1']).std()
std_amt_spent = std_amt_spent.rename(columns={'TransactionAmt':'std_amt_spent'})
test_transaction = test_transaction.merge(std_amt_spent, on='card1', how='left').reset_index()
del std_amt_spent

average amount spent on **card1** by using card type **card6**

In [ ]:
avg_amt_card_type = test_transaction[['card6', 'card1', 'TransactionAmt']].groupby(by=['card6', 'card1'])['TransactionAmt'].mean().reset_index()
test_transaction['avg_amt_card_type'] = avg_amt_card_type['TransactionAmt']

avg_amt_card_type = train_transaction[['card6', 'card1', 'TransactionAmt']].groupby(by=['card6', 'card1'])['TransactionAmt'].mean().reset_index()
train_transaction['avg_amt_card_type'] = avg_amt_card_type['TransactionAmt']
del avg_amt_card_type

number of times card used today until current transaction using **card1**

In [ ]:
train_transaction['card_used'] = 1
train_transaction['card_used_till_now_today'] = train_transaction[['card1', 'day','card_used']].groupby(['card1', 'day'])['card_used'].apply(lambda x: x.cumsum())

In [ ]:
del train_transaction['card_used']

test_transaction['card_used'] = 1
test_transaction['card_used_till_now_today'] = test_transaction[['card1', 'day','card_used']].groupby(['card1', 'day'])['card_used'].apply(lambda x: x.cumsum())
del test_transaction['card_used']

In [ ]:
train = pd.merge(train_transaction,train_identity,on='TransactionID', how='left')
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

print('train dataset shape', train.shape)
print('test dataset shape', test.shape)

In [ ]:
test[:30].to_csv('test.csv', columns=test.columns)

In [ ]:
del train_identity, train_transaction, test_identity, test_transaction

In [ ]:
train.to_pickle('train.pkl')
test.to_pickle('test.pkl')